In [1]:
import sys
sys.executable

'D:\\Program Files\\Python\\Python36\\python.exe'

In [2]:
# 随机种子
seed = 2020

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
import time
import math
import utils
import random
from tqdm import tqdm
import logging
import numpy as np
import tqdm
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# 一、数据预处理

本次实验共有4类数据集，分为train/dev/test1/test2, 其中test2只有英文原文. 每一英文原文只有一句中文参考。
训练集train共有一千万条数据，考虑到机器性能及训练时间问题，我们随机选取500000条数据作为训练集。

In [6]:
src_path = "./nmt_dataset4389/train/train.en"
trg_path = "./nmt_dataset4389/train/train.zh"
np.random.seed(42)

with open(src_path, "r", encoding="utf-8") as f:
    src_line = []
    for line in tqdm.tqdm(f):
        src_line.append(line.replace("\n", ""))


with open(trg_path, "r", encoding="utf-8") as f:
    trg_line = []
    for line in tqdm.tqdm(f):
        trg_line.append(line.replace("\n", ""))

res = []
for i in range(len(src_line)):
    res.append([src_line[i],trg_line[i]])

np.random.shuffle(res)

name = ["en","zh"]
train=pd.DataFrame(columns=name,data=res)

train.to_csv('./nmt_dataset4389/train/train.csv',index=False)



10000000it [00:10, 948848.85it/s]
10000000it [00:13, 747610.95it/s]


## 1.1正则化

In [7]:
# 正则化，主要是将部分标点符号统一处理
import re
def normalizeString(s): 
    s = str(s)
    s = s.replace('，', ',')
    s = s.replace('？', '?')
    s = s.replace('！', '!')
    s = s.replace('：', ':')

    return s

## 1.2读取数据

### 1.2.1训练集

In [8]:
train_shuffle = pd.read_csv("./nmt_dataset4389/train/train.csv")[:500000]
en_data = []
ch_data = []
for i in range(len(train_shuffle)):
    en_data.append(normalizeString(train_shuffle["en"][i]))
    ch_data.append(normalizeString(train_shuffle["zh"][i]))
print('英文数据:\n', en_data[:10])
print('\n中文数据:\n', ch_data[:10])

英文数据:
 ['But I have very set ideas. ', "Glad your accounting degree hasn't gone to waste.", "They say it's because medicine attracts perfectionists.", 'Schmidt! Schmidt, I need you to help me ', "Despite it's vast size", 'A separate disease, acute oak decline, is killing oaks, while chestnut trees have been felled across the country because of infections spread by moths.', 'For now, even atms here are rare, as are places that accept credit cards.', 'I want to see you occasionally doing - great job. Very positive response.', 'Look, he was drunk. He apologized.', 'Whoa, Freddie, ']

中文数据:
 ['我有我的想法。 ', '你的会计学位没浪费还真让人高兴。', '他们说这是因为医学吸引完美主义者。', '施密特!施密特,你得帮帮我 ', '不管有多大', '另一种疾病急性栎树衰退病正在杀死大量的栎树,同时由于此病的传染性能借助蛾子传播,栗树也在全国范围内被砍伐。', '目前,缅甸就连自动柜员机都很少,更不用说受理信用卡的地点。', '我想看到你偶尔也能做做- 干得好。非常高的评价。', '你看,他喝醉了,他道歉了。', '喔,弗雷迪, ']


In [9]:
# 按字符级切割，并添加<eos>
en_token_list = [[char for char in line]+["<eos>"] for line in en_data]
ch_token_list = [[char for char in line]+["<eos>"] for line in ch_data]
print('英文数据:\n', en_token_list[:2])
print('\n中文数据:\n', ch_token_list[:2])

英文数据:
 [['B', 'u', 't', ' ', 'I', ' ', 'h', 'a', 'v', 'e', ' ', 'v', 'e', 'r', 'y', ' ', 's', 'e', 't', ' ', 'i', 'd', 'e', 'a', 's', '.', ' ', '<eos>'], ['G', 'l', 'a', 'd', ' ', 'y', 'o', 'u', 'r', ' ', 'a', 'c', 'c', 'o', 'u', 'n', 't', 'i', 'n', 'g', ' ', 'd', 'e', 'g', 'r', 'e', 'e', ' ', 'h', 'a', 's', 'n', "'", 't', ' ', 'g', 'o', 'n', 'e', ' ', 't', 'o', ' ', 'w', 'a', 's', 't', 'e', '.', '<eos>']]

中文数据:
 [['我', '有', '我', '的', '想', '法', '。', ' ', '<eos>'], ['你', '的', '会', '计', '学', '位', '没', '浪', '费', '还', '真', '让', '人', '高', '兴', '。', '<eos>']]


In [10]:
# 基本字典
basic_dict = {'<pad>':0, '<unk>':1, '<bos>':2, '<eos>':3}
# 分别生成中英文字典 

en_vocab = set(''.join(en_data))
en_vocab = sorted(en_vocab) # 保证每次结果都一样

en2id = {char:i+len(basic_dict) for i, char in enumerate(en_vocab)}

en2id.update(basic_dict)

id2en = {v:k for k,v in en2id.items()}

# 分别生成中英文字典 
ch_vocab = set(''.join(ch_data))
ch_vocab = sorted(ch_vocab) # 保证每次结果都一样

ch2id = {char:i+len(basic_dict) for i, char in enumerate(ch_vocab)}
ch2id.update(basic_dict)
id2ch = {v:k for k,v in ch2id.items()}



In [11]:
# 利用字典，映射数据 
en_num_data = [[en2id[en] for en in line ] for line in en_token_list]
ch_num_data = [[ch2id[ch] for ch in line] for line in ch_token_list]

print('char:', en_data[1])
print('index:', en_num_data[1])

char: Glad your accounting degree hasn't gone to waste.
index: [44, 81, 70, 73, 5, 94, 84, 90, 87, 5, 70, 72, 72, 84, 90, 83, 89, 78, 83, 76, 5, 73, 74, 76, 87, 74, 74, 5, 77, 70, 88, 83, 12, 89, 5, 76, 84, 83, 74, 5, 89, 84, 5, 92, 70, 88, 89, 74, 19, 3]


### 1.2.2验证集

In [12]:
from xml.dom.minidom import parse
import xml.dom.minidom

src_path = "./nmt_dataset4389/dev/dev.en-zh.en.xml"
trg_path = "./nmt_dataset4389/dev/dev.en-zh.zh.xml"
    
dev_en_data = []
dev_ch_data = []
# 使用minidom解析器打开 XML 文档
DOMTree_src = xml.dom.minidom.parse(src_path)
collection = DOMTree_src.documentElement
sentences = collection.getElementsByTagName("seg")
for sentence in sentences:
    dev_en_data.append(normalizeString(sentence.childNodes[0].data))

DOMTree_trg = xml.dom.minidom.parse(trg_path)
collection = DOMTree_trg.documentElement
sentences = collection.getElementsByTagName("seg")
for sentence in sentences:
    dev_ch_data.append(normalizeString(sentence.childNodes[0].data))

print('英文数据:\n', dev_en_data[:10])
print('\n中文数据:\n', dev_ch_data[:10])

英文数据:
 [' Because you want to start a family before you hit the nursing home. ', " She's got to have me in her sight like 24 hours a day. ", " So that no parent has to go through what I've known. ", " Is when someone we've trusted makes the choice for us. ", ' I respect that, and I will protect it at all cost. ', ' So after investigators got a blood trace on the doorknob, ', " Which means if we don't find her, she's only got two weeks to live. ", " Because you need to know that whoever you're protecting is damn sure gonna be selfish ", ' And you can also see the theater staircase very well preserved; ', " Cos that's not what I'm hearing. Still, what do I know? "]

中文数据:
 [' 因为你想在进养老院前娶妻生子。 ', ' 我就一天24小时都得在她眼皮子底下。 ', ' 为了不让别的父母经历我的遭遇。 ', ' 有时候我们信任的人替我们做了这样的选择。 ', ' 我尊重这点,并且会不惜一切保护隐私不被侵犯。 ', ' 所以在调查了有微量血迹的门把手之后, ', ' 如果我们找不到她,她就只有两周可活了。 ', ' 因为你要知道不管你在保护谁等你们一起进了号子 ', ' 你们还可以看到剧场台阶保存得十分完好； ', ' 因为就我所听到的可不是。不过,我懂什么啊? ']


In [13]:
# 按字符级切割，并添加<eos>
dev_en_token_list = [[char for char in line]+["<eos>"] for line in dev_en_data]
dev_ch_token_list = [[char for char in line]+["<eos>"] for line in dev_ch_data]
print('英文数据:\n', dev_en_token_list[:2])
print('\n中文数据:\n', dev_ch_token_list[:2])

英文数据:
 [[' ', 'B', 'e', 'c', 'a', 'u', 's', 'e', ' ', 'y', 'o', 'u', ' ', 'w', 'a', 'n', 't', ' ', 't', 'o', ' ', 's', 't', 'a', 'r', 't', ' ', 'a', ' ', 'f', 'a', 'm', 'i', 'l', 'y', ' ', 'b', 'e', 'f', 'o', 'r', 'e', ' ', 'y', 'o', 'u', ' ', 'h', 'i', 't', ' ', 't', 'h', 'e', ' ', 'n', 'u', 'r', 's', 'i', 'n', 'g', ' ', 'h', 'o', 'm', 'e', '.', ' ', '<eos>'], [' ', 'S', 'h', 'e', "'", 's', ' ', 'g', 'o', 't', ' ', 't', 'o', ' ', 'h', 'a', 'v', 'e', ' ', 'm', 'e', ' ', 'i', 'n', ' ', 'h', 'e', 'r', ' ', 's', 'i', 'g', 'h', 't', ' ', 'l', 'i', 'k', 'e', ' ', '2', '4', ' ', 'h', 'o', 'u', 'r', 's', ' ', 'a', ' ', 'd', 'a', 'y', '.', ' ', '<eos>']]

中文数据:
 [[' ', '因', '为', '你', '想', '在', '进', '养', '老', '院', '前', '娶', '妻', '生', '子', '。', ' ', '<eos>'], [' ', '我', '就', '一', '天', '2', '4', '小', '时', '都', '得', '在', '她', '眼', '皮', '子', '底', '下', '。', ' ', '<eos>']]


In [14]:
# 利用字典，映射数据 
dev_en_num_data = [[en2id[en] if en in en2id else basic_dict["<unk>"] for en in line ] for line in dev_en_token_list]
dev_ch_num_data = [[ch2id[ch] if ch in ch2id  else basic_dict["<unk>"] for ch in line] for line in dev_ch_token_list]

print('char:', dev_en_data[1])
print('index:', dev_en_num_data[1])

char:  She's got to have me in her sight like 24 hours a day. 
index: [5, 56, 77, 74, 12, 88, 5, 76, 84, 89, 5, 89, 84, 5, 77, 70, 91, 74, 5, 82, 74, 5, 78, 83, 5, 77, 74, 87, 5, 88, 78, 76, 77, 89, 5, 81, 78, 80, 74, 5, 23, 25, 5, 77, 84, 90, 87, 88, 5, 70, 5, 73, 70, 94, 19, 5, 3]


### 1.2.3测试集1

In [15]:
from xml.dom.minidom import parse
import xml.dom.minidom

src_path = "./nmt_dataset4389/test1/test1.en-zh.en.xml"
trg_path = "./nmt_dataset4389/test1/test1.en-zh.zh.xml"
    
test1_en_data = []
test1_ch_data = []
# 使用minidom解析器打开 XML 文档
DOMTree_src = xml.dom.minidom.parse(src_path)
collection = DOMTree_src.documentElement
sentences = collection.getElementsByTagName("seg")
for sentence in sentences:
    test1_en_data.append(normalizeString(sentence.childNodes[0].data))

DOMTree_trg = xml.dom.minidom.parse(trg_path)
collection = DOMTree_trg.documentElement
sentences = collection.getElementsByTagName("seg")
for sentence in sentences:
    test1_ch_data.append(normalizeString(sentence.childNodes[0].data))

print('英文数据:\n', test1_en_data[:10])
print('\n中文数据:\n', test1_ch_data[:10])

英文数据:
 [" Hi, honey. I guess you're really tied up in meetings. ", ' Find a safety chain or something to keep these lights in place. ', " Okay. Well, I guess there's not much to do about it right now then. ", " Yeah, it's getting weird. - let's get out of here. ", ' If you want to remain team manager, you have to do this. ', ' I was the same way after I lost the baby. ', ' His legitimate business brings in what? Five million a year to us? ', ' Bring it home, kid! The finish line is wide open! ', " You know I'm paying for these tickets. It's all right. I got lunch. ", ' The two with the lowest number of votes will be leaving us ']

中文数据:
 [' 嗨,亲爱的。你现在肯定忙着开会呢。 ', ' 找条牢靠的链子或者别的什么固定住这些灯。 ', ' 好吧。那么,我想现在能做的有限。 ', ' 太奇怪了。- 我们出去吧。 ', ' 如果你还想做球队经理的话你必须得做这个。 ', ' 我失去孩子后也这样过。 ', ' 他的合法业务带给我们多少收入?一年五百万? ', ' 孩子,加油!就要到了终点了! ', ' 你要知道付机票钱的可是我那好吧,我请你吃午餐。 ', ' 得票最低的两位选手将在周六晚的1小时揭晓节目 ']


In [16]:
# 按字符级切割，并添加<eos>
test1_en_token_list = [[char for char in line]+["<eos>"] for line in test1_en_data]
test1_ch_token_list = [[char for char in line]+["<eos>"] for line in test1_ch_data]
print('英文数据:\n', test1_en_token_list[:2])
print('\n中文数据:\n', test1_ch_token_list[:2])

英文数据:
 [[' ', 'H', 'i', ',', ' ', 'h', 'o', 'n', 'e', 'y', '.', ' ', 'I', ' ', 'g', 'u', 'e', 's', 's', ' ', 'y', 'o', 'u', "'", 'r', 'e', ' ', 'r', 'e', 'a', 'l', 'l', 'y', ' ', 't', 'i', 'e', 'd', ' ', 'u', 'p', ' ', 'i', 'n', ' ', 'm', 'e', 'e', 't', 'i', 'n', 'g', 's', '.', ' ', '<eos>'], [' ', 'F', 'i', 'n', 'd', ' ', 'a', ' ', 's', 'a', 'f', 'e', 't', 'y', ' ', 'c', 'h', 'a', 'i', 'n', ' ', 'o', 'r', ' ', 's', 'o', 'm', 'e', 't', 'h', 'i', 'n', 'g', ' ', 't', 'o', ' ', 'k', 'e', 'e', 'p', ' ', 't', 'h', 'e', 's', 'e', ' ', 'l', 'i', 'g', 'h', 't', 's', ' ', 'i', 'n', ' ', 'p', 'l', 'a', 'c', 'e', '.', ' ', '<eos>']]

中文数据:
 [[' ', '嗨', ',', '亲', '爱', '的', '。', '你', '现', '在', '肯', '定', '忙', '着', '开', '会', '呢', '。', ' ', '<eos>'], [' ', '找', '条', '牢', '靠', '的', '链', '子', '或', '者', '别', '的', '什', '么', '固', '定', '住', '这', '些', '灯', '。', ' ', '<eos>']]


In [17]:
# 利用字典，映射数据 
test1_en_num_data = [[en2id[en] if en in en2id else basic_dict["<unk>"] for en in line ] for line in test1_en_token_list]
test1_ch_num_data = [[ch2id[ch] if ch in ch2id  else basic_dict["<unk>"] for ch in line] for line in test1_ch_token_list]

print('char:', test1_en_data[1])
print('index:', test1_en_num_data[1])

char:  Find a safety chain or something to keep these lights in place. 
index: [5, 43, 78, 83, 73, 5, 70, 5, 88, 70, 75, 74, 89, 94, 5, 72, 77, 70, 78, 83, 5, 84, 87, 5, 88, 84, 82, 74, 89, 77, 78, 83, 76, 5, 89, 84, 5, 80, 74, 74, 85, 5, 89, 77, 74, 88, 74, 5, 81, 78, 76, 77, 89, 88, 5, 78, 83, 5, 85, 81, 70, 72, 74, 19, 5, 3]


### 1.2.4测试集2

In [18]:
from xml.dom.minidom import parse
import xml.dom.minidom

src_path = "./nmt_dataset4389/test2/test2.en-zh.en.xml"

test2_en_data = []

# 使用minidom解析器打开 XML 文档
DOMTree_src = xml.dom.minidom.parse(src_path)
collection = DOMTree_src.documentElement
sentences = collection.getElementsByTagName("seg")
for sentence in sentences:
    test2_en_data.append(normalizeString(sentence.childNodes[0].data))


print('英文数据:\n', test2_en_data[:10])


英文数据:
 [' Do you think we look young enough to blend in at a high school? ', ' I have to go to the date, learn to dance. Definitely. Now. ', " But, still, I'm starting to think it would be smart ", " Okay? You wing your ass down here and you tell him I don't know. ", " You gonna take care of his kids, huh, after he's gone? ", " I am not going back for more and I can't even believe ", ' Where is he? Get on. Get on the fucking ground! ', " And I already have a place, okay? It's at your place. ", ' And that every young man who chooses to remain at home ', ' I left my bag of chargers at home. Did I? ']


In [19]:
# 按字符级切割，并添加<eos>
test2_en_token_list = [[char for char in line]+["<eos>"] for line in test2_en_data]

print('英文数据:\n', test2_en_token_list[:2])


英文数据:
 [[' ', 'D', 'o', ' ', 'y', 'o', 'u', ' ', 't', 'h', 'i', 'n', 'k', ' ', 'w', 'e', ' ', 'l', 'o', 'o', 'k', ' ', 'y', 'o', 'u', 'n', 'g', ' ', 'e', 'n', 'o', 'u', 'g', 'h', ' ', 't', 'o', ' ', 'b', 'l', 'e', 'n', 'd', ' ', 'i', 'n', ' ', 'a', 't', ' ', 'a', ' ', 'h', 'i', 'g', 'h', ' ', 's', 'c', 'h', 'o', 'o', 'l', '?', ' ', '<eos>'], [' ', 'I', ' ', 'h', 'a', 'v', 'e', ' ', 't', 'o', ' ', 'g', 'o', ' ', 't', 'o', ' ', 't', 'h', 'e', ' ', 'd', 'a', 't', 'e', ',', ' ', 'l', 'e', 'a', 'r', 'n', ' ', 't', 'o', ' ', 'd', 'a', 'n', 'c', 'e', '.', ' ', 'D', 'e', 'f', 'i', 'n', 'i', 't', 'e', 'l', 'y', '.', ' ', 'N', 'o', 'w', '.', ' ', '<eos>']]


In [20]:
# 利用字典，映射数据 
test2_en_num_data = [[en2id[en] if en in en2id else basic_dict["<unk>"] for en in line ] for line in test2_en_token_list]


print('char:', test2_en_data[1])
print('index:', test2_en_num_data[1])

char:  I have to go to the date, learn to dance. Definitely. Now. 
index: [5, 46, 5, 77, 70, 91, 74, 5, 89, 84, 5, 76, 84, 5, 89, 84, 5, 89, 77, 74, 5, 73, 70, 89, 74, 17, 5, 81, 74, 70, 87, 83, 5, 89, 84, 5, 73, 70, 83, 72, 74, 19, 5, 41, 74, 75, 78, 83, 78, 89, 74, 81, 94, 19, 5, 51, 84, 92, 19, 5, 3]


## 1.3表示为Dataset

In [21]:
class TranslationDataset(Dataset):
    def __init__(self, src_data, trg_data):
        self.src_data = src_data
        self.trg_data = trg_data

        assert len(src_data) == len(trg_data), \
            "numbers of src_data  and trg_data must be equal!"

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        src_sample =self.src_data[idx]
        src_len = len(self.src_data[idx])
        trg_sample = self.trg_data[idx]
        trg_len = len(self.trg_data[idx])
        return {"src": src_sample, "src_len": src_len, "trg": trg_sample, "trg_len": trg_len}

## 1.4填充处理

In [22]:
def padding_batch(batch):
    """
    input: -> list of dict
        [{'src': [1, 2, 3], 'trg': [1, 2, 3]}, {'src': [1, 2, 2, 3], 'trg': [1, 2, 2, 3]}]
    output: -> dict of tensor 
        {
            "src": [[1, 2, 3, 0], [1, 2, 2, 3]].T
            "trg": [[1, 2, 3, 0], [1, 2, 2, 3]].T
        }
    """
    src_lens = [d["src_len"] for d in batch]
    trg_lens = [d["trg_len"] for d in batch]
    
    src_max = max([d["src_len"] for d in batch])
    trg_max = max([d["trg_len"] for d in batch])
    for d in batch:
        d["src"].extend([en2id["<pad>"]]*(src_max-d["src_len"]))
        d["trg"].extend([ch2id["<pad>"]]*(trg_max-d["trg_len"]))
    srcs = torch.tensor([pair["src"] for pair in batch], dtype=torch.long, device=device)
    trgs = torch.tensor([pair["trg"] for pair in batch], dtype=torch.long, device=device)
    
    batch = {"src":srcs.T, "src_len":src_lens, "trg":trgs.T, "trg_len":trg_lens}
    return batch

# 二、模型搭建

## 2.1Encoder

In [23]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout=0.5, bidirectional=True):
        super(Encoder, self).__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout, bidirectional=bidirectional)
        
    def forward(self, input_seqs, input_lengths, hidden):
        # input_seqs = [seq_len, batch]
        embedded = self.embedding(input_seqs)
        # embedded = [seq_len, batch, embed_dim]
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, enforce_sorted=False)
        
        outputs, hidden = self.gru(packed, hidden)        
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # outputs = [seq_len, batch, hid_dim * n directions]
        # output_lengths = [batch]
        return outputs, hidden

## 2.2Attention

In [24]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)  # [seq_len, batch]

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)  # [seq_len, batch, hid_dim]
        return torch.sum(hidden * energy, dim=2)  # [seq_len, batch]

    def concat_score(self, hidden, encoder_output):
        # hidden.expand(encoder_output.size(0), -1, -1) -> [seq_len, batch, N]
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        # energy = [sql_len, batch, hidden_size]
        return torch.sum(self.v * energy, dim=2)  # [seq_len, batch]

    def forward(self, hidden, encoder_outputs):
        # hidden = [1, batch,  n_directions * hid_dim]
        # encoder_outputs = [seq_len, batch, hid dim * n directions]
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        attn_energies = attn_energies.t()  # [batch, seq_len]
 
        return F.softmax(attn_energies, dim=1).unsqueeze(1)  # softmax归一化# [batch, 1, seq_len]

## 2.3Decoder

In [25]:
class AttnDecoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers=1, dropout=0.5, bidirectional=True, attn_method="general"):
        super(AttnDecoder, self).__init__()

        self.output_dim = output_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout, bidirectional=bidirectional)
        
        if bidirectional:
            self.concat = nn.Linear(hid_dim * 2 * 2, hid_dim*2)
            self.out = nn.Linear(hid_dim*2, output_dim)
            self.attn = Attn(attn_method, hid_dim*2)
        else:
            self.concat = nn.Linear(hid_dim * 2, hid_dim)
            self.out = nn.Linear(hid_dim, output_dim)
            self.attn = Attn(attn_method, hid_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, token_inputs, last_hidden, encoder_outputs):
        batch_size = token_inputs.size(0)
        embedded = self.embedding(token_inputs)
        embedded = self.embedding_dropout(embedded)
        embedded = embedded.view(1, batch_size, -1) # [1, B, hid_dim],其中1为seq_len

        gru_output, hidden = self.gru(embedded, last_hidden)
        # gru_output = [1, batch,  n_directions * hid_dim]
        # hidden = [n_layers * n_directions, batch, hid_dim]

        # encoder_outputs = [sql_len, batch, hid dim * n directions]
        attn_weights = self.attn(gru_output, encoder_outputs)
        # attn_weights = [batch, 1, sql_len]
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # [batch, 1, hid_dim * n directions]

        # LuongAttention
        gru_output = gru_output.squeeze(0) # [batch, n_directions * hid_dim]
        context = context.squeeze(1)       # [batch, n_directions * hid_dim]
        concat_input = torch.cat((gru_output, context), 1)  # [batch, n_directions * hid_dim * 2]
        concat_output = torch.tanh(self.concat(concat_input))  # [batch, n_directions*hid_dim]

        output = self.out(concat_output)  # [batch, output_dim]
        output = self.softmax(output)

        return output, hidden, attn_weights

## 2.4Seq2seq

In [26]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 device, 
                 predict=False, 
                 basic_dict=None,
                 max_len=100
                 ):
        super(Seq2Seq, self).__init__()
        
        self.device = device

        self.encoder = encoder
        self.decoder = decoder

        self.predict = predict  # 训练阶段还是预测阶段
        self.basic_dict = basic_dict  # decoder的字典，存放特殊token对应的id
        self.max_len = max_len  # 翻译时最大输出长度

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        assert encoder.gru.bidirectional == decoder.gru.bidirectional, \
            "Decoder and encoder must had same value of bidirectional attribute!"
        
    def forward(self, input_batches, input_lengths, target_batches=None, target_lengths=None, teacher_forcing_ratio=0.5):
        # input_batches = [seq_len, batch]
        # target_batches = [seq_len, batch]
        batch_size = input_batches.size(1)
        
        BOS_token = self.basic_dict["<bos>"]
        EOS_token = self.basic_dict["<eos>"]
        PAD_token = self.basic_dict["<pad>"]

        # 初始化
        enc_n_layers = self.encoder.gru.num_layers
        enc_n_directions = 2 if self.encoder.gru.bidirectional else 1
        encoder_hidden = torch.zeros(enc_n_layers*enc_n_directions, batch_size, self.encoder.hid_dim, device=self.device)
        
        # encoder_outputs = [input_lengths, batch, hid_dim * n directions]
        # encoder_hidden = [n_layers*n_directions, batch, hid_dim]
        encoder_outputs, encoder_hidden = self.encoder(
            input_batches, input_lengths, encoder_hidden)

        # 初始化
        decoder_input = torch.tensor([BOS_token] * batch_size, dtype=torch.long, device=self.device)
        decoder_hidden = encoder_hidden

        if self.predict:
            # 一次只输入一句话
            assert batch_size == 1, "batch_size of predict phase must be 1!"
            output_tokens = []

            while True:
                decoder_output, decoder_hidden, decoder_attn = self.decoder(
                    decoder_input, decoder_hidden, encoder_outputs
                )
                # [1, 1]
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(1).detach()
                output_token = topi.squeeze().detach().item()
                if output_token == EOS_token or len(output_tokens) == self.max_len:
                    break
                output_tokens.append(output_token)
            return output_tokens

        else:
            max_target_length = max(target_lengths)
            all_decoder_outputs = torch.zeros((max_target_length, batch_size, self.decoder.output_dim), device=self.device)

            for t in range(max_target_length):
                use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
                if use_teacher_forcing:
                    # decoder_output = [batch, output_dim]
                    # decoder_hidden = [n_layers*n_directions, batch, hid_dim]
                    decoder_output, decoder_hidden, decoder_attn = self.decoder(
                        decoder_input, decoder_hidden, encoder_outputs
                    )
                    all_decoder_outputs[t] = decoder_output
                    decoder_input = target_batches[t]  # 下一个输入来自训练数据
                else:
                    decoder_output, decoder_hidden, decoder_attn = self.decoder(
                        decoder_input, decoder_hidden, encoder_outputs
                    )
                    # [batch, 1]
                    topv, topi = decoder_output.topk(1)
                    all_decoder_outputs[t] = decoder_output
                    decoder_input = topi.squeeze(1).detach()  # 下一个输入来自模型预测
            
            loss_fn = nn.NLLLoss(ignore_index=PAD_token)
            loss = loss_fn(
                all_decoder_outputs.reshape(-1, self.decoder.output_dim),  # [batch*seq_len, output_dim]
                target_batches.reshape(-1)               # [batch*seq_len]
            )
            return loss

# 三、训练和预测代码

In [27]:
class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [28]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [29]:
def train(
    writer,
    model,
    data_loader, 
    optimizer, 
    clip=1, 
    teacher_forcing_ratio=0.5, 
    print_every=1000  # None不打印
    ):
    
    global global_step
    global best_valid_loss
    global best_dev_loss
    
    model.predict = False
    

    if print_every == 0:
        print_every = 1

    print_loss_total = 0  # 每次打印都重置
    start = time.time()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
  
        model.train()

        # shape = [seq_len, batch]
        input_batchs = batch["src"]
        target_batchs = batch["trg"]
        # list
        input_lens = batch["src_len"]
        target_lens = batch["trg_len"]
        
        optimizer.zero_grad()
        try:
            loss = model(input_batchs, input_lens, target_batchs, target_lens, teacher_forcing_ratio)
  
            print_loss_total += loss.item()
            epoch_loss += loss.item()
            loss.backward()
        except RuntimeError as e:
            if 'out of memory' in str(e):
                print('| WARNING: ran out of memory')
                if hasattr(torch.cuda, 'empty_cache'):
                    torch.cuda.empty_cache()
            else:
                raise e

        # 梯度裁剪
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        global_step += 1
        # Logging
        writer.add_scalar('Train/Loss', loss.item(), global_step)
 
        optimizer.step()
    
        if print_every and global_step % print_every == 0:
            test_one_sample()
            valid_loss = evaluate(model, dev_loader)
            es(valid_loss, model, model_path="en2ch-attn-model.pt")
            '''
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                torch.save(model.state_dict(), "en2ch-attn-model.pt")
            '''
        
            writer.add_scalar('Eval/Loss', valid_loss, global_step)
      

        if print_every and global_step % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('\tCurrent Loss: %.4f' % print_loss_avg)
            if print_loss_avg < best_dev_loss:
                best_dev_loss = print_loss_avg
                torch.save(model.state_dict(), "en2ch-attn-model_train.pt")

    return epoch_loss / len(data_loader)

In [30]:
def evaluate(
    model,
    data_loader, 
    print_every=None
    ):
    model.predict = False
    model.eval()
    if print_every == 0:
        print_every = 1

    print_loss_total = 0  # 每次打印都重置
    start = time.time()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
           
            # shape = [seq_len, batch]
            input_batchs = batch["src"]
            target_batchs = batch["trg"]
            # list
            input_lens = batch["src_len"]
            target_lens = batch["trg_len"]
            try:
                loss = model(input_batchs, input_lens, target_batchs, target_lens, teacher_forcing_ratio=0)
            except RuntimeError as e:
                if 'out of memory' in str(e):
                    print('| WARNING: ran out of memory')
                    if hasattr(torch.cuda, 'empty_cache'):
                          torch.cuda.empty_cache()
                else:
                    raise e
            print_loss_total += loss.item()
            epoch_loss += loss.item()

            if print_every and (i+1) % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('\tCurrent Loss: %.4f' % print_loss_avg)
    
    msg = f'*** Eval: loss {epoch_loss / len(data_loader)}'
    logger.info(msg)

    return epoch_loss / len(data_loader)

In [31]:
def translate(
    model,
    sample, 
    idx2token=None
    ):
    model.predict = True
    model.eval()

    # shape = [seq_len, 1]
    input_batch = sample["src"]
    # list
    input_len = sample["src_len"]

    output_tokens = model(input_batch, input_len)
    output_tokens = [idx2token[t] for t in output_tokens]

    return "".join(output_tokens)

# 四、超参数配置

In [32]:
INPUT_DIM = len(en2id)
OUTPUT_DIM = len(ch2id)
# 超参数
BATCH_SIZE = 16
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
LEARNING_RATE = 1e-4
N_EPOCHS = 200
CLIP = 1

bidirectional = True
attn_method = "general"
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT, bidirectional)
dec = AttnDecoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, bidirectional, attn_method)
model = Seq2Seq(enc, dec, device, basic_dict=basic_dict).to(device)


optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [33]:
# 数据集
train_set = TranslationDataset(en_num_data, ch_num_data)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=padding_batch)

dev_set = TranslationDataset(dev_en_num_data, dev_ch_num_data)
dev_loader = DataLoader(dev_set, batch_size=BATCH_SIZE, collate_fn=padding_batch)

test1_set = TranslationDataset(test1_en_num_data, test1_ch_num_data)
test1_loader = DataLoader(test1_set, batch_size=BATCH_SIZE, collate_fn=padding_batch)



# 五、开始训练

In [ ]:
best_valid_loss = float('inf')
best_dev_loss = float('inf')
global_step = 0
writer = SummaryWriter()
es = EarlyStopping(patience=20, mode="min")
model.load_state_dict(torch.load("en2ch-attn-model_train.pt"))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    tq_train = tqdm(train_loader, total=len(train_loader))
    train_loss = train(writer,model, tq_train, optimizer, CLIP)
    valid_loss = evaluate(model, dev_loader)
    #dev_loss = evaluate(model, tq_train)
    end_time = time.time()
    '''
    if dev_loss < best_dev_loss:
        best_dev_loss = dev_loss
        torch.save(model.state_dict(), "en2ch-attn-model_train.pt")
    '''
    
    es(valid_loss, model, model_path="en2ch-attn-model.pt")

    if epoch %1 == 0:
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

    torch.cuda.empty_cache()
    
    

# 六、BLEU评价测试集

In [34]:
#print("best valid loss：", best_valid_loss)
# 加载最优权重
model.load_state_dict(torch.load("en2ch-attn-model.pt"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [35]:
random.seed(seed)
for i in random.sample(range(len(en_num_data)), 10):  # 随机看10个
    en_tokens = list(filter(lambda x: x!=0, en_num_data[i]))  # 过滤零
    ch_tokens = list(filter(lambda x: x!=3 and x!=0, ch_num_data[i]))  # 和机器翻译作对照
    sentence = [id2en[t] for t in en_tokens]
    print("【原文】")
    print("".join(sentence))
    translation = [id2ch[t] for t in ch_tokens]
    print("【原文】")
    print("".join(translation))
    test_sample = {}
    test_sample["src"] = torch.tensor(en_tokens, dtype=torch.long, device=device).reshape(-1, 1)
    test_sample["src_len"] = [len(en_tokens)]
    print("【机器翻译】")
    print(translate(model, test_sample, id2ch), end="\n\n")

【原文】
Someone pulled him out.<eos>
【原文】
有人把他拉出来的。
【机器翻译】
有人把他出出来。

【原文】
Uh, what happened to your small bowel obstruction?<eos>
【原文】
额,你的小肠梗阻病人怎么样了?
【机器翻译】
呃,你的小小小的小小的小小?

【原文】
I thought I'd catch you on the way in. I'd love the opportunity to discuss... <eos>
【原文】
希望有机会。能和您坐下聊聊... 
【机器翻译】
我以为我会在你你上面。我会喜欢这个机会... 

【原文】
Her Bush looks like the hedge maze<eos>
【原文】
她的阴毛看上去像是“闪灵”里面的
【机器翻译】
她的脸子看起来像是一样的

【原文】
Alert all ground crews. <eos>
【原文】
所有地面人员。 
【机器翻译】
所有的地方都是。 

【原文】
Kiss me! Kiss him.<eos>
【原文】
吻我!吻他。
【机器翻译】
吻吻我!吻他。

【原文】
Force at backrest = <eos>
【原文】
靠背的支撑力= 
【机器翻译】
在背景的 

【原文】
I almost lost control of those feelings.<eos>
【原文】
我几乎失控。
【机器翻译】
几乎失败了我控制了。

【原文】
Call me a traitor again, love, and I'll end you.<eos>
【原文】
再叫我叛徒小妞,我立马杀了你。
【机器翻译】
再叫叛叛叛徒,我会你你。

【原文】
With who? Who cares? <eos>
【原文】
找谁啊?谁都行? 
【机器翻译】
哪个?谁?谁? 



## 6.1test1 bleu结果和平均得分

In [36]:
score = 0
score_store = []
for i in range(len(test1_en_num_data)):
    en_tokens = list(filter(lambda x: x!=0, test1_en_num_data[i]))  # 过滤零
    ch_tokens = list(filter(lambda x: x!=3 and x!=0, test1_ch_num_data[i]))  # 和机器翻译作对照
    sentence = [id2en[t] for t in en_tokens]
    print("【原文】")
    print("".join(sentence))
    translation = [id2ch[t] for t in ch_tokens]
    print("【原文】")
    print("".join(translation))
    test_sample = {}
    test_sample["src"] = torch.tensor(en_tokens, dtype=torch.long, device=device).reshape(-1, 1)
    test_sample["src_len"] = [len(en_tokens)]
    print("【机器翻译】")
    print(translate(model, test_sample, id2ch), end="\n\n")
    temp = sentence_bleu(["".join(translation)],translate(model, test_sample, id2ch))
    score += temp
    score_store.append(temp)
    
score /= len(test1_en_num_data)
    
print(score)
    

【原文】
 Hi, honey. I guess you're really tied up in meetings. <eos>
【原文】
 嗨,亲爱的。你现在肯定忙着开会呢。 
【机器翻译】
嗨,嗨,亲爱的。我想你你真的在会议。 

【原文】
 Find a safety chain or something to keep these lights in place. <eos>
【原文】
 找条牢靠的链子或者别的什么固定住这些灯。 
【机器翻译】
找到一个安全的或许是让这些灯子里的东西。 

【原文】
 Okay. Well, I guess there's not much to do about it right now then. <eos>
【原文】
 好吧。那么,我想现在能做的有限。 
【机器翻译】
好吧。那我想我们没有么多事。 



D:\Program Files\Python\Python36\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


【原文】
 Yeah, it's getting weird. - let's get out of here. <eos>
【原文】
 太奇怪了。- 我们出去吧。 
【机器翻译】
是啊,怪了。- 我们走吧。 

【原文】
 If you want to remain team manager, you have to do this. <eos>
【原文】
 如果你还想做球队经理的话你必须得做这个。 
【机器翻译】
如果你想要一个队伍,你必须这么做。 

【原文】
 I was the same way after I lost the baby. <eos>
【原文】
 我失去孩子后也这样过。 
【机器翻译】
我是我失去了一个失去的 

【原文】
 His legitimate business brings in what? Five million a year to us? <eos>
【原文】
 他的合法业务带给我们多少收入?一年五百万? 
【机器翻译】
他的意思是什么事情的意思??五千万年? 

【原文】
 Bring it home, kid! The finish line is wide open! <eos>
【原文】
 孩子,加油!就要到了终点了! 
【机器翻译】
把它带回家,孩子!! 

D:\Program Files\Python\Python36\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)




【原文】
 You know I'm paying for these tickets. It's all right. I got lunch. <eos>
【原文】
 你要知道付机票钱的可是我那好吧,我请你吃午餐。 
【机器翻译】
你知道我在付票票。没事。我午饭了。 

【原文】
 The two with the lowest number of votes will be leaving us <eos>
【原文】
 得票最低的两位选手将在周六晚的1小时揭晓节目 
【机器翻译】
两位票票将我们最低的票票 

【原文】
 So I let her believe that she was getting to me. <eos>
【原文】
 所以我让她相信她说服我了。 
【机器翻译】
所以我让她相信她我我我。 

【原文】
 May I have a word, please? What do you want? <eos>
【原文】
 我可以说句话吗?你要干什么? 
【机器翻译】
我能有个词吗?你想要什么? 

【原文】
 He found it the night the president went to the national archives to view the constitution, <eos>
【原文】
 总统去国家档案馆参观宪法的那晚他发现的, 
【机器翻译】
他在国国家里找到到了国国的国家的宪法, 

【原文】
 I took the best offer while I had a chance. <eos>
【原文】
 当我有机会时我接受了最好的邀约。 
【机器翻译】
我有机会的时候我我了。 

【原文】
 That was an awful lot of trouble just to impress your fans. <eos>
【原文】
 只是想在你粉丝面前露一手可费了不少劲儿啊。 
【机器翻译】
你这么多的的你想象你的粉丝。 

【原文】
 At the turn of the tide, water from almost seven hundred miles. <eos>
【原文】
 当潮水转向时,从近700里外峡湾而来的水。 
【机器翻译】
从最近的的的的的,水里, 

【原文】
 It made me 

...

【原文】
 I can't wait to check this place out. - Yeah. <eos>
【原文】
 我要马上到处逛逛- 是啊。 
【机器翻译】
我不能等这个地方去查查这个地方。- 是的。 

【原文】
 But you can only lose it for you, nobody else. <eos>
【原文】
 但你要只为自己而减而不是为别人。 
【机器翻译】
但是你只能给你丢下去,别人人。 

【原文】
 Yeah, don't take somebody out of rehab before the rehab! <eos>
【原文】
 对,别在复健之前将人带离复健! 
【机器翻译】
是的,别人别人在休息前休息! 

【原文】
 Ye - so maybe he asked a guard to take the gun and the cell phone if you did. <eos>
【原文】
 所以也许他告诉保安一旦你杀了他就拿走枪和手机。 
【机器翻译】
如果他问了你个人,请拿枪枪手机,打电话给他。 

【原文】
 You're awfully confident for someone who's got no cards left to play. <eos>
【原文】
 对于一个无牌可出的人你也太自信了。 
【机器翻译】
你对于有人不会的人的人不会有人的的人。 

【原文】
 Both black men. Both times he was cleared, but only just. <eos>
【原文】
 两次他射的都是黑人两次他都无罪,但都是差一点。 
【机器翻译】
两个男人都黑人都清楚了,他他都清醒了。 

【原文】
 You and me ain't never gonna happen - never! Ya hear me?! <eos>
【原文】
 你跟我永远没戏--- 永远!听见了没?! 
【机器翻译】
你没有永远不会发生的!听到我? 

【原文】
 If you understood him, you wouldn't subject him to ridicule. <eos>
【原文】
 如果你了解他,你不会嘲笑他。 
【机器翻译】
如果你能解他他,你不会他他他会拒绝他。 



【原文】
 But, Jas, you said boys don't rate girls for funniness. <eos>
【原文】
 但是佳斯你说过男孩们不喜欢女孩搞笑。 
【机器翻译】
但是,杰斯,你说孩子们不不会女孩们不会不会。 

【原文】
 Good, because I think I have someone you just might be interested in. <eos>
【原文】
 很好,因为我觉得我手头有个人你应该会感兴趣的。 
【机器翻译】
很好,因为我觉得你有人有兴趣。 

【原文】
 Yeah, but isn't it bad luck to buy a dress for the victory party before the election? <eos>
【原文】
 但是在竞选结果出来之前就为他的庆功宴买新衣会不会不大好? 
【机器翻译】
是的,但是,但是没有么对选举的选举的选举? 

【原文】
 If I don't, Neither of us will ever speak again <eos>
【原文】
 如果我不说,我们两个以后就不能再说话了 
【机器翻译】
如果我不不会,我们会不会再说话 

【原文】
 What do you mean they don't know what wrong with it? <eos>
【原文】
 什么叫他们也不清楚咋回事? 
【机器翻译】
你什么意思是他们不知道怎么办? 

【原文】
 For this round, you must bounce the ball on the drum, over a series of obstacles. <eos>
【原文】
 在这一轮里,你们要用鼓弹球突破系列障碍。 
【机器翻译】
你必须把这个轮,你必须绕上一个。 

【原文】
 You know, if you watched the local news like a normal person, <eos>
【原文】
 你知道,如果你像正常人一样看本地新闻, 
【机器翻译】
你知道,如果你看着当地的新闻的正常人, 

【原文】
 I mean, come on, baby. We are living the dream. <eos>
【原文】
 我是说,

哦,噢,不。你们回到同一个人 

【原文】
 But I need to get my father off my back. <eos>
【原文】
 但得先打发掉我父亲。 
【机器翻译】
但我得把我爸爸回来。 

【原文】
 We may have passion but passion does not win wars! <eos>
【原文】
 我们虽有激情但空有激情是打不赢战争的! 
【机器翻译】
我们可能会热情但是不过,但是我们可能会激情! 

【原文】
 Take Highway 151 to Dodgeville Exit 47 Highway 18 then go north one mile to John's Street. <eos>
【原文】
 沿151号高速公路到达道奇维尔,从47号出口退出到达18号高速公路,再往北走一英里到达约翰街。 
【机器翻译】
沿着15号公路向北驶向北驶向北驶向驶驶向北驶驶向北驶向驶驶驶驶驶驶驶驶驶驶驶驶驶驶驶驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。驶。 

【原文】
 By the 1 300s, the Templars had grown too powerful. <eos>
【原文】
 到了十三世纪武士团的势力已经过于壮大。 
【机器翻译】
十十00年代,,庙墓的已经有力量。 

【原文】
 A few months ago, there were two men here taking pictures. <eos>
【原文】
 几月前,两个男人在这拍照。 
【机器翻译】
几个月前,有两个人人照片。 

【原文】
 Do you think he'd sign a copy for me? Um, yes. <eos>
【原文】
 你认为他能否给我签名?恩,是的。 
【机器翻译】
你觉得他会给我找个吗?嗯,是的。 

【原文】
 You guys showed an incredible amount of sophistication in your movement <eos>
【原文】
 你们的舞步中表现出了难以置信的成熟 
【机器翻译】
你们出现了一个的的的的的的 

【原文】
 Nice. That's what I'm talkin

你想说什么?你想想个?想打架我??你想打架我? 

【原文】
 There is no way that this thing is gonna fit - <eos>
【原文】
 如果林肯都装不了这个箱子 - 
【机器翻译】
不过,这不能这样的东西- 

【原文】
 He wants no wire mesh and no rabbits. How can we do it? <eos>
【原文】
 他不想要铁丝网和兔子。我们怎么办? 
【机器翻译】
他想不会不会不是不是不是不是我兔子。我们怎么能做到? 

【原文】
 How are you liking working here so far, aside from the misogynistic bits and pieces? <eos>
【原文】
 你对工作还满意吗,除了那些闲言碎语外? 
【机器翻译】
你们这样的工作如何远远,,,远离开这里的? 

【原文】
 Fuck, now they're harassing me with a dying old man. <eos>
【原文】
 他妈的,现在他们让个快死的老头来烦我! 
【机器翻译】
他妈,现在他们我们人一起了一个人。 

【原文】
 If I join them, maybe I can find out who. <eos>
【原文】
 我加入他们,或许就能找出那个叛徒。 
【机器翻译】
如果我能加入他们,也许我能找到谁找到的。 

【原文】
 It's secluded, and I swear nobody can find you there. <eos>
【原文】
 与世隔绝,我发誓不会有人找到你的。 
【机器翻译】
我已经被困扰了,我发现你没人能找到到你。 

【原文】
 But if you're ever in the mood for a crossover, step - back, fade away "J" in your face, <eos>
【原文】
 但如果哪天你有兴致,转身过人,后撤步,后仰跳投, 
【机器翻译】
但是如果你是过你的过你的的话, 

【原文】
 A few years ago, I was hanging out in a gas station bathroom <eos>
【原文】
 几

【原文】
 Who it belongs to, stuff like that. it's mine. No, it's mine... <eos>
【原文】
 两个人在吵那车应该归谁是我的。不,是我的…… 
【机器翻译】
他属于属于属于属于这是我的属于我的。不是,我是我的... 

【原文】
 Was one of the best meals we've had in a long time. <eos>
【原文】
 是这么久以来最美味的。 
【机器翻译】
我们一直很久的时候我们最好的一 

【原文】
 Not with this girl's blood on his hands, he won't. <eos>
【原文】
 但双手沾满女孩的血印的话,那就悬了。 
【机器翻译】
他不是这个女孩的血手,不会。 

【原文】
 I'm going to close my eyes and pretend like I was there. <eos>
【原文】
 我要闭上眼先幻想下自己身临其境。 
【机器翻译】
我要闭眼睛睛后我要闭我眼睛。 

【原文】
 Extortion? Not even you would sink that low. Sure I would. <eos>
【原文】
 敲诈?就算你也不会下作到这种程度吧。我当然会。 
【机器翻译】
出口??不是你会那么做。我会的。 

【原文】
 I'd heard the price of eggs was going up, but isn't that a little high? <eos>
【原文】
 我听说珠宝蛋的价格已飚涨,但这不是贵了一点吗? 
【机器翻译】
我就会听到那些鸡蛋的鸡蛋,但是不是是个小高? 

【原文】
 I can hear your thoughts. Yes, and I can hear yours. <eos>
【原文】
 我可以听到你的想法。是的,我也可以听到你的。 
【机器翻译】
我能听到你的思想。听到你的思想。听到了。 

【原文】
 I don't take orders from you! I take orders from Saddam Hussain. <eos>
【原文】
 我不听从你的命令!我听从的是萨达姆•侯赛因的命令。 
【机器翻译】
我

## 6.2test1 5个较低分数的翻译结果对比

In [37]:
import heapq
result = map(score_store.index, heapq.nsmallest(5, score_store))
for r in result:
    print("bleu得分：",score_store[r])
    en_tokens = list(filter(lambda x: x!=0, test1_en_num_data[r]))  # 过滤零
    ch_tokens = list(filter(lambda x: x!=3 and x!=0, test1_ch_num_data[r]))  # 和机器翻译作对照
    sentence = [id2en[t] for t in en_tokens]
    print("【原文】")
    print("".join(sentence))
    translation = [id2ch[t] for t in ch_tokens]
    print("【原文】")
    print("".join(translation))
    test_sample = {}
    test_sample["src"] = torch.tensor(en_tokens, dtype=torch.long, device=device).reshape(-1, 1)
    test_sample["src_len"] = [len(en_tokens)]
    print("【机器翻译】")
    print(translate(model, test_sample, id2ch), end="\n\n")
    

bleu得分： 3.9210590692235436e-232
【原文】
 He waited for it to surface through a hole in the ice <eos>
【原文】
 他一直等待看那只海豹从冰面的褒窿钻出来 
【机器翻译】
他在冰冰里面着, 

bleu得分： 4.369817601630177e-232
【原文】
 I was on my way to work, when - bang, bang, bang! <eos>
【原文】
 当时我正在上班路上,突然就听到砰,砰,砰! 
【机器翻译】
我在在我班班班,, 

bleu得分： 5.177746370966683e-232
【原文】
 You did tell them he attacked you? Oh, what I would give <eos>
【原文】
 你也说了遭袭的事?哦我告诉他们一切 
【机器翻译】
你跟他打了你?? 

bleu得分： 5.786230302664252e-232
【原文】
 To go to a tax - free state and end up selling your business <eos>
【原文】
 去一个免税国家结束自己的销售业务 
【机器翻译】
去买税务,免费, 

bleu得分： 6.05006004817715e-232
【原文】
 We have had banquets in the headquarters in these nights <eos>
【原文】
 秘密基地这几天晚上都在办大食会 
【机器翻译】
今晚我们有有有的队 



## 6.3test1 5个较高分数的翻译结果对比

In [38]:
import heapq
result = map(score_store.index, heapq.nlargest(5, score_store))

for r in result:
    print("bleu得分：",score_store[r])
    en_tokens = list(filter(lambda x: x!=0, test1_en_num_data[r]))  # 过滤零
    ch_tokens = list(filter(lambda x: x!=3 and x!=0, test1_ch_num_data[r]))  # 和机器翻译作对照
    sentence = [id2en[t] for t in en_tokens]
    print("【原文】")
    print("".join(sentence))
    translation = [id2ch[t] for t in ch_tokens]
    print("【原文】")
    print("".join(translation))
    test_sample = {}
    test_sample["src"] = torch.tensor(en_tokens, dtype=torch.long, device=device).reshape(-1, 1)
    test_sample["src_len"] = [len(en_tokens)]
    print("【机器翻译】")
    print(translate(model, test_sample, id2ch), end="\n\n")
    

bleu得分： 0.7710792371922995
【原文】
 I'm sorry. I know you're just trying to protect him. <eos>
【原文】
 抱歉。我知道你只是想保护他。 
【机器翻译】
抱歉,我知道你只是想保护他。 

bleu得分： 0.762465858623486
【原文】
 It's because she's your sister and you have a chance to help her. <eos>
【原文】
 因为她是你妹妹你有机会帮她。 
【机器翻译】
因为她是你妹妹而且你有机会帮她。 

bleu得分： 0.7105477917053865
【原文】
 What did you just say? - what did you just say? <eos>
【原文】
 你刚才说什么?-你刚才说什么? 
【机器翻译】
你刚才说什么?- 你刚刚说什么? 

bleu得分： 0.6422727797927412
【原文】
 We can all go out together. No. This is a date. <eos>
【原文】
 我们可以一起去。不,这是约会。 
【机器翻译】
我们可以去一起去。不。这是约会。 

bleu得分： 0.6380018521198061
【原文】
 We have included a microwave, refrigerator, coffee maker, hairdryer and iron. <eos>
【原文】
 我们的客房配备微波炉、冰箱、咖啡机、吹风机和熨斗。 
【机器翻译】
我们配备包括一台微波炉、冰箱、咖啡机、吹风机、吹风机和熨斗。 



## 6.3test2 结果保存

In [39]:
def updateXML():
    domTree = parse("./nmt_dataset4389/test2/test2.en-zh.en.xml")
    # 文档根元素
    rootNode = domTree.documentElement

    segs = rootNode.getElementsByTagName("seg")

    for i in range(len(test2_en_num_data)):
        en_tokens = list(filter(lambda x: x != 0, test2_en_num_data[i]))  # 过滤零
        sentence = [id2en[t] for t in en_tokens]
        # print("【原文】")
        # print("".join(sentence))
        test_sample = {}
        test_sample["src"] = torch.tensor(en_tokens, dtype=torch.long, device=device).reshape(-1, 1)
        test_sample["src_len"] = [len(en_tokens)]
        # print("【机器翻译】")
        temp = translate(model, test_sample, id2ch)
        # print(temp, end = "\n\n")
        segs[i].childNodes[0].data = temp


    with open('updated_customer.xml', 'w') as f:
        # 缩进 - 换行 - 编码
        domTree.writexml(f, addindent='  ', encoding='gb2312')

if __name__ == '__main__':
	updateXML()

# 七、attention对比

## 7.1global attention 与 soft attention 结构完全一样

In [40]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)  # [seq_len, batch]

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)  # [seq_len, batch, hid_dim]
        return torch.sum(hidden * energy, dim=2)  # [seq_len, batch]

    def concat_score(self, hidden, encoder_output):
        # hidden.expand(encoder_output.size(0), -1, -1) -> [seq_len, batch, N]
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        # energy = [sql_len, batch, hidden_size]
        return torch.sum(self.v * energy, dim=2)  # [seq_len, batch]

    def forward(self, hidden, encoder_outputs):
        # hidden = [1, batch,  n_directions * hid_dim]
        # encoder_outputs = [seq_len, batch, hid dim * n directions]
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        attn_energies = attn_energies.t()  # [batch, seq_len]
 
        return F.softmax(attn_energies, dim=1).unsqueeze(1)  # softmax归一化# [batch, 1, seq_len]

## 7.2hard attention 

Hard Attention，这种方式是直接精准定位到某个key，其余key就都不管了，相当于这个key的概率是1，其余key的概率全部是0。因此这种对齐方式要求很高，要求一步到位，如果没有正确对齐，会带来很大的影响。另一方面，因为不可导，一般需要用强化学习的方法进行训练。（或者使用gumbel softmax之类的）

## 7.3local attention

In [41]:
class LocalAttn(nn.Module):
    def __init__(self, method, hidden_size):
        super(LocalAttn, self).__init__()
        self.D = 5
        self.window_len = 2 * self.D + 1
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        self.Wp = nn.Linear(self.hidden_size, hidden_size)
        self.Vp = nn.Parameter(torch.FloatTensor(hidden_size))

        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        # hidden.shape =  [1, batch,  n_directions * hid_dim]
        # encoder_output.shape = [seq_len, batch,  n_directions * hid_dim]
        return torch.sum(hidden * encoder_output, dim=2)  # [seq_len, batch]

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)  # [seq_len, batch, hid_dim]
        return torch.sum(hidden * energy, dim=2)  # [seq_len, batch]

    def concat_score(self, hidden, encoder_output):
        # hidden.expand(encoder_output.size(0), -1, -1) -> [seq_len, batch, N]
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        # energy = [sql_len, batch, hidden_size]
        return torch.sum(self.v * energy, dim=2)  # [seq_len, batch]

    def calculate_pt(self,hidden,S):
        # hidden.squeeze(0).shape = [batch,  n_directions * hid_dim]
        # self.Wp(hidden.squeeze(0)).tanh().shape = [batch, hidden_size]
        # self.Wp(hidden.squeeze(0)).tanh()*self.Vp.shape = [batch, hidden_size]
        # torch.sum(self.Wp(hidden.squeeze(0)).tanh()*self.Vp, dim=1).shape = [batch]

        return S * torch.sum(self.Wp(hidden.squeeze(0)).tanh()*self.Vp, dim=1).sigmoid()

    def calculate_exp(self, t, Pt, D):
        # Pt.shape = [batch]
        t = torch.tensor([t] * Pt.shape[0])

        return torch.exp(-(t-Pt)**2/2/((D/2)**2))


    def forward(self, hidden, encoder_outputs,t):
        # hidden = [1, batch,  n_directions * hid_dim]
        # encoder_outputs = [seq_len, batch, hid dim * n directions]
        S = encoder_outputs.shape[0]
        Pt = self.calculate_pt(hidden,S)  # [batch]
        exp = self.calculate_exp(t,Pt,self.D)  # [batch]

        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)  # [seq_len,batch]

        attn_energies *= exp

        attn_energies = attn_energies.t()  # [batch, seq_len]

        return F.softmax(attn_energies, dim=1).unsqueeze(1)  # softmax归一化# [batch, 1, seq_len]
